## Import Libraries

In [1]:
from genericpath import exists
import pdb
from pickletools import uint8
from turtle import pd
import torchvision
import torch


import os
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
# from torchknickknacks import modelutils
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from pytorch_grad_cam.utils.image import show_cam_on_image

import numpy as np
import cv2
import requests
from pytorch_grad_cam.ablation_layer import AblationLayerFasterRCNN
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget
from pytorch_grad_cam.utils.reshape_transforms import fasterrcnn_reshape_transform
from pytorch_grad_cam.ablation_layer import AblationLayerFasterRCNN
import random
import glob
from skimage.measure import label, regionprops, regionprops_table
from skimage import data, filters, measure, morphology
import pandas as pd
import wandb
from tqdm import tqdm
from skimage import data
from skimage.color import rgb2hed, hed2rgb
import sys
sys.path.insert(0, '../')
from models.model_mrcnn import _default_mrcnn_config, build_default

/home/mahirwar/anaconda3/envs/amy_plague/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve

## Utility Functions

In [3]:
class ExplainPredictions():
    
    # TODO fix the visualization flags
    def __init__(self, model, model_input_path, test_input_path, detection_threshold, wandb, save_result, ablation_cam, save_thresholds):
        self.model = model
        self.model_input_path = model_input_path
        self.test_input_path = test_input_path
        self.detection_threshold = detection_threshold
        self.wandb = wandb
        self.save_result = save_result
        self.ablation_cam = ablation_cam
        self.save_thresholds = save_thresholds
        self.class_names = ['Unknown', 'Core', 'Diffuse', 'CAA']
        self.class_to_colors = {'Core': (255, 0, 0), 'Diffuse': (0,255,0), 'CAA':(225, 255, 0)}
        #TODO change this to nas location later
        #self.result_save_dir= "/wynton/home/finkbeiner/vgramas/Projects/amyb-plaque-detection/reports/figures/"
        self.result_save_dir= "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/Results/"
        self.colors = np.random.uniform(0, 255, size=(len(self.class_names), 3))
        self.column_names = ["image_name", "region", "region_mask", "label", 
                            "confidence", "brown_pixels", "centroid", 
                            "eccentricity", "area", "equivalent_diameter"]
        self.results_path = ""
        self.masks_path = "" 
        self.detections_path = "" 
        self.ablations_path = ""
        self.quantify_path = ""
      
    def get_brown_pixel_cnt(self, img, img_name):

        # Separate the stains from the IHC image
        ihc_hed = rgb2hed(img)

        # Create an RGB image for each of the stains
        null = np.zeros_like(ihc_hed[:, :, 0])
        ihc_h = hed2rgb(np.stack((ihc_hed[:, :, 0], null, null), axis=-1))
        ihc_e = hed2rgb(np.stack((null, ihc_hed[:, :, 1], null), axis=-1))
        ihc_d = hed2rgb(np.stack((null, null, ihc_hed[:, :, 2]), axis=-1))
        ihc_d = ihc_d.astype('float32')

        gray = cv2.cvtColor(ihc_d, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        x = gray[gray<0.35]


        if len(x) != 0:
            if self.save_thresholds:
                # Display
                fig, axes = plt.subplots(2, 2, figsize=(7, 6), sharex=True, sharey=True)
                ax = axes.ravel()

                ax[0].imshow(img)
                ax[0].set_title("Original image")

                ax[1].imshow(ihc_h)
                ax[1].set_title("Hematoxylin")

                ax[2].imshow(gray)
                ax[2].set_title("gray")  # Note that there is no Eosin stain in this image

                ax[3].imshow(ihc_d)
                ax[3].set_title("DAB")

                for a in ax.ravel():
                    a.axis('off')

                fig.tight_layout()
            
                
                final_save_path = img_name + "_count_threhold.png"
                final_save_path = os.path.join(self.pixel_count_path, final_save_path)
                fig.savefig(final_save_path)
                plt.close()
            return len(x)
        return 0


    def get_outputs(self, input_tensor, model, threshold):
        with torch.no_grad():
            # forward pass of the image through the modle
            outputs = model(input_tensor)
        
        # get all the scores
        scores = list(outputs[0]['scores'].detach().cpu().numpy())
        # print("\n scores", max(scores))
        # index of those scores which are above a certain threshold
        thresholded_preds_inidices = [scores.index(i) for i in scores if i > threshold]
        thresholded_preds_count = len(thresholded_preds_inidices)

        scores = scores[:thresholded_preds_count]
        # get the masks
        masks = (outputs[0]['masks']>0.5).squeeze().detach().cpu().numpy()
        # print("masks", masks)
        # discard masks for objects which are below threshold
        masks = masks[:thresholded_preds_count]
        # get the bounding boxes, in (x1, y1), (x2, y2) format
        boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in outputs[0]['boxes'].detach().cpu()]
        # discard bounding boxes below threshold value
        boxes = boxes[:thresholded_preds_count]
        # get the classes labels
        # print('labels', outputs[0]['labels'])
        labels = [self.class_names[i] for i in outputs[0]['labels']]
        labels = labels[:thresholded_preds_count]

        # [1,1,1, 2, 2, 2, 3, 3]
        return masks, boxes, labels, scores

    def draw_segmentation_map(self, image, masks, boxes, labels):
        alpha = 1 
        beta = 0.6 # transparency for the segmentation map
        gamma = 0 # scalar added to each 
        segmentation_map = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        result_masks = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        for i in range(len(masks)):

            # TODO fix the color segmentation masks
            red_map = np.zeros_like(masks[i]).astype(np.uint8)
            green_map = np.zeros_like(masks[i]).astype(np.uint8)
            blue_map = np.zeros_like(masks[i]).astype(np.uint8)
            
            # apply a randon color mask to each object
            rect_color = (0,0,0)
            color = self.colors[random.randrange(0, len(self.colors))]
            red_map[masks[i] == 1], green_map[masks[i] == 1], blue_map[masks[i] == 1]  = self.class_to_colors[labels[i]]
            result_masks[masks[i] == 1] = 255
            # combine all the masks into a single image
            # change the format of mask to W,H, C

            # segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
            #convert the original PIL image into NumPy format
            image = np.array(image)
            # convert from RGB to OpenCV BGR format
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # apply mask on the image
            # cv2.addWeighted(image, alpha, segmentation_map, beta, gamma, image)
            # draw the bounding boxes around the objects
            cv2.rectangle(image, boxes[i][0], boxes[i][1], color=rect_color, 
                        thickness=2)
            # Get the centre coords of the rectangle-plaque-detection/src/visualizat
            x1 = boxes[i][0][0]
            y1 = boxes[i][0][1]
            x2 = boxes[i][1][0]
            y2 = boxes[i][1][1]
            x = int((x1 + x2) / 2)
            y = int((y1+y2) / 2)

            
            # put the label text above the objects
            cv2.putText(image , labels[i], (x1, y1-20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 
                        thickness=2, lineType=cv2.LINE_AA)
            
            # Convert Back
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image, result_masks

    def prepare_input(self, image):
    
        image_float_np = np.float32(image) / 255

        # define the torchvision image transforms
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(1024),
            torchvision.transforms.ToTensor(),
        ])

        
        input_tensor = transform(transforms.ToPILImage()(image))
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_tensor = input_tensor.to(device)
        # Add a batch dimension:
        input_tensor = input_tensor.unsqueeze(0)

        return input_tensor, image_float_np
    
    def predict(self, input_tensor, model, device, detection_threshold):
        outputs = model(input_tensor)
        # i- 1 zero indexing - the model outputs a non zero indexing format ( 1, 2, 3)
        pred_classes = [self.class_names[i] for i in outputs[0]['labels'].cpu().numpy()]
        pred_labels = outputs[0]['labels'].cpu().numpy()
        pred_scores = outputs[0]['scores'].detach().cpu().numpy()
        pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
        
        boxes, classes, labels, indices = [], [], [], []
        for index in range(len(pred_scores)):
            if pred_scores[index] >= detection_threshold:
                boxes.append(pred_bboxes[index].astype(np.int32))
                classes.append(pred_classes[index])
                labels.append(pred_labels[index])
                indices.append(index)
        boxes = np.int32(boxes)
        return boxes, classes, labels, indices

    def draw_boxes(self, boxes, labels, classes, image):
        for i, box in enumerate(boxes):
            color = self.colors[labels[i]]
            cv2.rectangle(
                image,
                (int(box[0]), int(box[1])),
                (int(box[2]), int(box[3])),
                color, 2
            )
            cv2.putText(image, classes[i], (int(box[0]), int(box[1] + 30)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1,
                        lineType=cv2.LINE_AA)
        return image

    def make_result_dirs(self, folder_name):

        # TODO remove this later and replace with self.wandb.name
        # folder_name = self.wandb.name + "_" + folder_name
        folder_name = "runtest" + "_" + folder_name
        save_path = os.path.join(self.result_save_dir, folder_name)
        results_path = os.path.join(save_path, "results")
        if not os.path.exists(results_path):
            os.makedirs(results_path)
        
        detections_path = os.path.join(save_path, "detections")
        if not os.path.exists(detections_path):
            os.makedirs(detections_path)
        
        masks_path = os.path.join(save_path, "masks")
        if not os.path.exists(masks_path):
            os.makedirs(masks_path)
        
        ablations_path = os.path.join(save_path, "ablations")
        if not os.path.exists(ablations_path):
            os.makedirs(ablations_path)
        
        pixel_count_path = os.path.join(self.result_save_dir, "pixel_count")
        if not os.path.exists(pixel_count_path):
            os.makedirs(pixel_count_path)
        
        csv_name = folder_name + "_quantify.csv"
        quantify_path = os.path.join(save_path, csv_name)

        self.results_path = results_path
        self.masks_path = masks_path
        self.detections_path = detections_path
        self.ablations_path = ablations_path
        self.quantify_path = quantify_path
        self.pixel_count_path = pixel_count_path

    def quantify_plaques(self, df, wandb_result, img_name, result_img, result_masks, boxes, labels, scores, total_brown_pixels):
        '''This function will take masks image and generate attributes like plaque
        count, area, eccentricity'''

        csv_result = []
        
    
        for i in range(len(labels)):

            props = {}
            data = {}
            # Here x and y axis are flipped
            total_core_plaques = 0
            total_neuritic_plaques = 0
            total_diffused_plaques = 0

            if len(boxes)!= 0:

                x1 = boxes[i][0][1]
                x2 =  boxes[i][1][1]
                y1 = boxes[i][0][0]
                y2 = boxes[i][1][0]

               
                cropped_img = result_img[x1:x2, y1:y2]
                cropped_img_mask = result_masks[x1:x2, y1:y2]

                ret, bw_img = cv2.threshold(cropped_img_mask,0,255,cv2.THRESH_BINARY)

                kernel = np.ones((5,5),np.uint8)
                
                # Closing operation Dilation followed by erosion
                closing = cv2.morphologyEx(bw_img, cv2.MORPH_CLOSE, kernel)
                regions = regionprops(closing)

                for props in regions:

                    if labels[i] == "Core":
                        total_core_plaques+=1
                    elif labels[i] == "Neuritic":
                        total_neuritic_plaques+=1
                    elif labels[i] == "Diffuse":
                        total_diffused_plaques+=1
                    
                    data_record = pd.DataFrame.from_records([{ 'image_name': img_name, 'label': labels[i] , 'confidence': scores[i],
                                                               'brown_pixels': total_brown_pixels,
                                                               'core': total_core_plaques, 'neuritic': total_neuritic_plaques, 'diffuse': total_diffused_plaques,
                                                               'centroid': props.centroid, 'eccentricity': props.eccentricity, 
                                                               'area': props.area, 'equivalent_diameter': props.equivalent_diameter}])
                    wandb_result.append([img_name, wandb.Image(cropped_img), wandb.Image(cropped_img_mask), labels[i], scores[i], 
                                         total_brown_pixels, props.centroid, props.eccentricity, props.area, props.equivalent_diameter])

                    df = pd.concat([df, data_record], ignore_index=True)
                   
        
        return df, wandb_result
 
    def generate_results(self):
        # This will help us create a different color for each class
        # Load Trained 
        
        self.model.load_state_dict(torch.load(self.model_input_path))
    
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.eval().to(device)
        output_dict=dict()
        output_score=dict()
        

        test_folders = glob.glob(os.path.join(self.test_input_path, "*"))
        
        # Test images from each WSI folder
        test_folders = sorted(test_folders)
        
        print("length of test folders", len(test_folders))
        
        output_dict = dict()
      
        for test_folder in tqdm(test_folders):

            print("\n", test_folder)
            
            folder_name = os.path.basename(test_folder)

            if folder_name == "labels":
                continue

            # make all necessary folders
            self.make_result_dirs(folder_name)
            images = glob.glob(os.path.join(test_folder, '*.jpg'))
            
            print("Images in Test Folder", len(images))
            i = 0
            df = pd.DataFrame()
            wandb_result = []
            total_core_plaques = 0
            total_neuritic_plaques = 0
            total_diffused_plaques = 0
            total_brown_pixels = 0
            total_image_pixels = 0

            for img in tqdm(images):
                result_img = 0
                img_name = os.path.basename(img).split('.')[0]
                
                image = np.array(Image.open(img))

                total_image_pixels+= image.shape[0] * image.shape[1]
                # Check if image has alpha channel
                if image.shape[2] == 4:
                    image = image[:,:, :3]
                input_tensor, image_float_np = self.prepare_input(image)

                masks, boxes, labels, scores = self.get_outputs(input_tensor, self.model, self.detection_threshold)
                #print(labels, scores)
                #print(labels)
                output_dict[img_name] = labels
                output_score[img_name]=scores
        return output_dict, output_score
    
    def generate_results_per_image(self):
        # This will help us create a different color for each class
        # Load Trained 
        output_dict=dict()
        output_score=dict()
        
        self.model.load_state_dict(torch.load(self.model_input_path))
    
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.eval().to(device)
        output_dict=dict()
        output_score=dict()
        test_folders = glob.glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/tiles/train/NA4751-02_AB", "*.jpg"))
        print(len(test_folders))
        #for Finkbeiner-Steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/tiles/train/NA4751-02_AB 
        for img in  test_folders:
            #img = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/tiles/train/NA4751-02_AB/NA4751-02_AB_19_6_34.jpg"
            #img = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/images/A_0_0_XE10-045_1_AmyB_1_49175x_54069y_image.png"
            img_name = os.path.basename(img).split('.')[0]
            image = np.array(Image.open(img))

            #total_image_pixels+= image.shape[0] * image.shape[1]
            # Check if image has alpha channel
            if image.shape[2] == 4:
                image = image[:,:, :3]
            input_tensor, image_float_np = self.prepare_input(image)

            masks, boxes, labels, scores = self.get_outputs(input_tensor, self.model, self.detection_threshold)
            #print(labels, scores)
            #print(labels)
            output_dict[img_name] = labels
            output_score[img_name]=scores
        return output_dict, output_score


## Evaluate External Dataset

In [11]:
image_dataset =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/tiles/train/"

#image_dataset =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/tiles/validation"

gt_train = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/classification_dataset/train.csv"

gt_val = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/tiles/validation.csv"

#model_input_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/models/peachy-firefly-487_mrcnn_model_50.pth"

model_input_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/models/eager-frog-489_mrcnn_model_100.pth"

#model_input_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/models/dry-disco-560_mrcnn_model_50.pth"

In [8]:
def evaluate_output(gt_train, output_df, output_score, train_flag):
    gt_train_df = pd.read_csv(gt_train)
    gt_train_df["imagename2"]= gt_train_df["imagename"].apply(lambda l: l.split("/")[-1].replace(".jpg",""))
    if train_flag:
        gt_train_df["cored_gt"] = gt_train_df["cored_bool"].apply(lambda l: 1 if l==True else 0)
        gt_train_df["diffuse_gt"] = gt_train_df["diffuse_bool"].apply(lambda l: 1 if l==True else 0)
        gt_train_df["CAA_gt"] = gt_train_df["CAA_bool"].apply(lambda l: 1 if l==True else 0)
    else:
        gt_train_df["cored_gt"] = gt_train_df["cored"].apply(lambda l: 1 if l>0 else 0)
        gt_train_df["diffuse_gt"] = gt_train_df["diffuse"].apply(lambda l: 1 if l>0 else 0)
        gt_train_df["CAA_gt"] = gt_train_df["CAA"].apply(lambda l: 1 if l>0 else 0)
    gt_train_df["cored_pred"] = 0
    gt_train_df["diffuse_pred"] = 0
    gt_train_df["CAA_pred"] = 0
    
    def assign_pred_scores(row, val):
        if row["imagename2"] in output_df.keys():
            l = output_df[row["imagename2"]]
            s = output_score[row["imagename2"]]
            indices = [i for i in range(len(l)) if l[i] == val]
            s_selected = [ s[i] for i in indices]
            if len(s_selected)!=0:
                return max(s_selected)
            else:
                return 0
        else:
            return None
    
    gt_train_df["cored_score"] = gt_train_df.apply(lambda l:  assign_pred_scores(l, "Core"), axis=1)
    gt_train_df["diffuse_score"] = gt_train_df.apply(lambda l:  assign_pred_scores(l, "Diffuse"),axis=1)
    gt_train_df["CAA_score"] = gt_train_df.apply(lambda l:  assign_pred_scores(l, "CAA"),axis=1)
    
    def assign_pred_labels(row, val):
        if row["imagename2"] in output_df.keys():
            l = output_df[row["imagename2"]]
            if val in l:
                return 1
            else:
                return 0
        else:
            return None
    
    gt_train_df["cored_pred"] = gt_train_df.apply(lambda l:  assign_pred_labels(l, "Core"), axis=1)
    gt_train_df["diffuse_pred"] = gt_train_df.apply(lambda l:  assign_pred_labels(l, "Diffuse"),axis=1)
    gt_train_df["CAA_pred"] = gt_train_df.apply(lambda l:  assign_pred_labels(l, "CAA"),axis=1)
    return gt_train_df
    

In [12]:
test_config = dict(
        batch_size = 1,
        num_classes = 3)
    
model_config = _default_mrcnn_config(num_classes=1 + test_config['num_classes']).config
model = build_default(model_config, im_size=1024)

In [ ]:
#run = wandb.init(project="LBD",  entity="monika-ahirwar")
run =""
explain = ExplainPredictions(model, model_input_path = model_input_path, test_input_path=image_dataset, 
                                detection_threshold=0.5, wandb=run, save_result=True, ablation_cam=False, save_thresholds=False)
output_df, output_scores = explain.generate_results()


In [9]:
# running validation dataset
gt_val_df = evaluate_output(gt_val, output_df, output_scores, False)

In [14]:
# running train dataset
gt_train_df = evaluate_output(gt_train, output_df, output_scores, False)

In [10]:
# excluding NA values from val
notna_gt_val_df = gt_val_df[~gt_val_df["cored_pred"].isna()]

In [15]:
# excluding NA values from train
notna_gt_train_df = gt_train_df[~gt_train_df["cored_pred"].isna()]

In [16]:
# dropping irrelevant columns from train
notna_gt_train_df.drop(['Unnamed: 0',"cored_bool","diffuse_bool","CAA_bool"], axis=1, inplace=True)

/tmp/ipykernel_88777/1838933179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notna_gt_train_df.drop(['Unnamed: 0',"cored_bool","diffuse_bool","CAA_bool"], axis=1, inplace=True)


In [17]:
# Concatenate train and val outputs
final_df = pd.concat([notna_gt_train_df,notna_gt_val_df], ignore_index=True)

In [18]:
final_df

,id,imagename,cored,diffuse,CAA,negative,flag,notsure,imagename2,cored_gt,diffuse_gt,CAA_gt,cored_pred,diffuse_pred,CAA_pred,cored_score,diffuse_score,CAA_score
0,10748,NA4757-02_AB/NA4757-02_AB_18_25_61.jpg,1.0,0.000000,0.0,0.0,0.0,0.0,NA4757-02_AB_18_25_61,1,0,0,1.0,0.0,0.0,0.856956,0.000000,0.00000
1,29503,NA4918-02_AB17-24/NA4918-02_AB17-24_9_18_12.jpg,0.0,2.832462,0.0,0.0,0.0,0.0,NA4918-02_AB17-24_9_18_12,0,1,0,0.0,1.0,0.0,0.000000,0.956901,0.00000
2,42524,NA4885-02_AB17-24/NA4885-02_AB17-24_4_23_50.jpg,1.0,1.000000,0.0,0.0,1.0,0.0,NA4885-02_AB17-24_4_23_50,1,1,0,1.0,0.0,0.0,0.992662,0.000000,0.00000
3,34432,NA4749-02_AB/NA4749-02_AB_17_12_50.jpg,0.0,3.770270,0.0,0.0,0.0,0.0,NA4749-02_AB_17_12_50,0,1,0,0.0,0.0,0.0,0.000000,0.000000,0.00000
4,4137,NA4751-02_AB/NA4751-02_AB_19_6_34.jpg,0.0,1.000000,2.0,0.0,2.0,0.0,NA4751-02_AB_19_6_34,0,1,1,0.0,0.0,1.0,0.000000,0.000000,0.98904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160621,0,NA_4933_02_AB17-24/neg_NA_4933_02_AB17-24_9_19...,0.0,0.000000,0.0,1.0,0.0,0.0,neg_NA_4933_02_AB17-24_9_19_0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00000
160622,0,NA_4933_02_AB17-24/neg_NA_4933_02_AB17-24_9_27...,0.0,0.000000,0.0,1.0,0.0,0.0,neg_NA_4933_02_AB17-24_9_27_4,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00000
160623,33873,NA_4933_02_AB17-24/NA_4933_02_AB17-24_9_3_1.jpg,0.0,1.000000,0.0,0.0,0.0,0.0,NA_4933_02_AB17-24_9_3_1,0,1,0,0.0,0.0,0.0,0.000000,0.000000,0.00000
160624,0,NA_4933_02_AB17-24/neg_NA_4933_02_AB17-24_9_3_...,0.0,0.000000,0.0,1.0,0.0,0.0,neg_NA_4933_02_AB17-24_9_3_4,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.00000


## Classification Metric - Core

In [19]:
print(classification_report(final_df["cored_gt"], final_df["cored_pred"]))

              precision    recall  f1-score   support

           0       0.85      0.64      0.73    109671
           1       0.49      0.76      0.60     50955

    accuracy                           0.68    160626
   macro avg       0.67      0.70      0.66    160626
weighted avg       0.74      0.68      0.69    160626



## Classification Metric - Diffuse

In [20]:
print(classification_report(final_df["diffuse_gt"], final_df["diffuse_pred"]))

              precision    recall  f1-score   support

           0       0.49      0.88      0.63     73941
           1       0.67      0.21      0.32     86685

    accuracy                           0.52    160626
   macro avg       0.58      0.55      0.47    160626
weighted avg       0.59      0.52      0.46    160626



## Classification Metric - CAA

In [21]:
print(classification_report(final_df["CAA_gt"], final_df["CAA_pred"]))

              precision    recall  f1-score   support

           0       0.76      0.99      0.86    110685
           1       0.96      0.29      0.44     49941

    accuracy                           0.78    160626
   macro avg       0.86      0.64      0.65    160626
weighted avg       0.82      0.78      0.73    160626



In [31]:
final_df.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/UCDavis-Dataset/classification_dataset/final_output_train_val.csv")

## ROC Curve

In [29]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc


colors={"cored":'royalblue', "diffuse":'firebrick','CAA':"green"}
df = final_df

fig = go.Figure()
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

for class_name in ["cored","diffuse","CAA"]:
    fpr, tpr, _ = roc_curve(df[class_name + "_gt"], df[class_name+"_score"])
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name=class_name, mode='lines',line=dict(color=colors[class_name], width=2)))

fig.update_layout(
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=700
)
fig.update_layout( plot_bgcolor='white', title="Receiver operating characteristic Curve")
fig.update_xaxes(mirror=True, ticks='outside', showline=True, linecolor='black',gridcolor='lightgrey')
fig.update_yaxes(mirror=True, ticks='outside', showline=True, linecolor='black',gridcolor='lightgrey')
fig.write_html("ROC_Curve.html")

## Precision Recall Curve

In [30]:
df = final_df

fig = go.Figure()
colors={"cored":'royalblue', "diffuse":'firebrick','CAA':"green"}
for class_name in ["cored","diffuse","CAA"]:
    pr, rc, _ = precision_recall_curve(df[class_name + "_gt"], df[class_name+"_score"])
    rp = (df[class_name + "_gt"]).sum()/len(df)
    fig.add_trace(go.Scatter(x=rc, y=pr, name=class_name, mode='lines',line=dict(color=colors[class_name], width=2)))
    #fig.add_trace(go.Scatter(x=[0,1], y=[rp,rp], name=class_name+"_random", mode='lines',line=dict(color=colors[class_name], width=2,dash='dash')))
    #fig.add_shape(type='line', line=dict(dash='dash',color=colors[class_name]), x0=0, x1=1, y0=rp, y1=rp, name=class_name)

fig.update_layout(
    xaxis_title='Recall',
    yaxis_title='Precision',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=700
)
fig.update_layout( plot_bgcolor='white',title="Precision-Recall Curve")
fig.update_xaxes(mirror=True, ticks='outside', showline=True, linecolor='black',gridcolor='lightgrey')
fig.update_yaxes(mirror=True, ticks='outside', showline=True, linecolor='black',gridcolor='lightgrey')
fig.write_html("PR_Curve.html")